In [1]:
import pandas as pd
import json
from utils import process_authors
from tqdm import tqdm
import pickle

In [2]:
data = pd.read_csv('../clustering/results/004_of_V13_topics.csv')

data.drop(data[data['authors'].map(lambda x: '\\' in r'%r' % x)].index, inplace=True)
data.drop(data[data['venue'].map(lambda x: '\\' in r'%r' % x)].index, inplace=True)
data = data.dropna(subset=['venue'])
data['venue'] = data['venue'].apply(lambda x: x.replace('None', 'null'))
data.drop(data[data['venue'].map(lambda x: x == '')].index, inplace=True)
data.drop(data[data['venue'].map(lambda x: '\\' in r'%r' % x)].index, inplace=True)
data['authors'] = data['authors'].apply(lambda x: json.loads(x.replace('\'', '\"')))
data['venue'] = data['venue'].apply(lambda x: json.loads(x.replace('\'', '\"')))
data['authors'] = data['authors'].apply(lambda x:  process_authors(x))
data.drop(data[data['authors'].map(lambda x: x == '[]')].index, inplace=True)

In [3]:
authors_id_dict = {}
with open('data/authors_id.txt', 'r') as f:
    for line in f:
        auhtor = line.strip().split()
        authors_id_dict[' '.join(auhtor[1:])] = int(auhtor[0])

In [4]:
from cogdl.oag import oagbert
tokenizer, model = oagbert("oagbert-v2")

author_oag_embedding_dict = {}
paper_oag_embedding_dict = {}
for i, row in tqdm(data.iterrows()):
    authors = []
    for author in row['authors']:
        if 'name' in author:
            authors.append(' '.join(author['name'].strip().split()))
    title = row['title']
    abstract = row['abstract']
    concepts = row['fos']
    if 'raw' in row['venue']:
        venue = row['venue']['raw']
    else:
        venue = ''
    try:
        paper_info = model.encode_paper(
        title=title,
        abstract=abstract,
        venue=venue,
        authors=authors,
        reduction="max",
        )
        
        paper_oag_embedding_dict[row['_id']] = paper_info['text'][0]['pooled_output'].detach().cpu()
        for author in paper_info['authors']:
            author_oag_embedding_dict[authors_id_dict[author['originalText']]] = author['pooled_output'].detach().cpu()
        # if i % 1000 == 0:
        #     with open(f'embeddings/authors_{i}.pkl', 'wb') as f:
        #         pickle.dump(author_oag_embedding_dict, f)
        #     with open(f'embeddings/papers_{i}.pkl', 'wb') as f:
        #         pickle.dump(paper_oag_embedding_dict, f)
    except ValueError:
        continue

51607it [07:39, 112.28it/s]


In [20]:
author_oag_embedding_dict_fixed = {}
authors_id_dict_fixed = {}
author_id_dict_inverse = {v: k for k, v in authors_id_dict.items()}

In [22]:
for i, j in enumerate(sorted(list(author_oag_embedding_dict.keys()))):
    author_oag_embedding_dict_fixed[i] = author_oag_embedding_dict[j]
    authors_id_dict_fixed[i] = author_id_dict_inverse[j]

In [29]:
with open('embeddings/authors.pkl', 'wb') as f:
    pickle.dump(author_oag_embedding_dict_fixed, f)
with open('embeddings/papers.pkl', 'wb') as f:
    pickle.dump(paper_oag_embedding_dict, f)

In [30]:
with open('data/idx2author.pkl', 'wb') as f:
    pickle.dump(authors_id_dict_fixed, f)
with open('data/author2idx.pkl', 'wb') as f:
    pickle.dump({v: k for k, v in authors_id_dict_fixed.items()}, f)